In [2]:
import classes
import time
import datetime
import pandas as pd
from tqdm import tqdm
import numpy


#open import data as pandas dataframe
patient_data = pd.read_csv("./data/PACJENT.TXT",sep=";",header=0)
test_data = pd.read_csv("./data/BADANIA2.TXT",sep=";",header=0)
bacteria_data = pd.read_csv("./data/DROBN.TXT",sep=";",header=0)
drag_sus_data = pd.read_csv("./data/LEKOWRAZ.TXT",sep=";",header=0)

In [3]:
print("Number of unfiltered patient records: " +  str(len(patient_data)))
print("Number of unfiltered tests records: " +  str(len(test_data)))
print("Number of unfiltered bacteria records: " +  str(len(bacteria_data)))
print("Number of unfiltered sus records: " +  str(len(drag_sus_data)))

Number of unfiltered patient records: 6861
Number of unfiltered tests records: 17198
Number of unfiltered bacteria records: 15553
Number of unfiltered sus records: 136286


In [4]:
test_classes = []

for index, data in test_data.iterrows():
    try:
        temp = classes.Test(data["badanie id"], data["id pobytu"], data["data"], data["kod materialu"])
        if temp.is_date_between( datetime.date(1992,1,1), datetime.date(1996,3,31)):
            test_classes.append(temp)
        else:
            pass
            #print("date outside of range at line: "+ str( index) )
            #print("date: " + str(temp.date) + "\n")
    except ValueError as e:
        print(str(e))
        print("bad value at line " + str(index) )
        continue 

year 199200 is out of range
bad value at line 1472
invalid literal for int() with base 10: '1993:00:00'
bad value at line 5849
year 19930 is out of range
bad value at line 5880
year 19940 is out of range
bad value at line 8777
year 19940 is out of range
bad value at line 10233
invalid literal for int() with base 10: '19950:00:00'
bad value at line 14552
invalid literal for int() with base 10: '19950:00:00'
bad value at line 14583


In [5]:
delta = len(test_data) - len(test_classes)
print("new test's lenght = {}, delta = {}".format(len(test_classes), delta))
    

new test's lenght = 16995, delta = 203


In [6]:
patient_classes = []
for index, data in patient_data.iterrows():
    temp_class = classes.Patient(data["numer"], data["sex"], data["depName"])
    if temp_class.PatientId not in [klass.PatientId for klass in patient_classes]:
        patient_classes.append(temp_class)
    else:
        print("same id at index: " + str(index))
        for i in range(len(patient_classes)):
            if temp_class.PatientId == patient_classes[i].PatientId:
                print("with id at: " + str(i))
                if temp_class.department[0] not in patient_classes[i].department:
                    patient_classes[i].extend_department(temp_class)
                    print(patient_classes[i].department)
                    break
print(len(patient_classes))

same id at index: 726
with id at: 725
['oiom', 'neur']
same id at index: 2225
with id at: 2223
['WEWN', 'nacz']
same id at index: 3326
with id at: 3323
['nefr', 'szcz']
same id at index: 3327
with id at: 3323
same id at index: 3670
with id at: 3665
['PLAS', 'urol']
same id at index: 3872
with id at: 3866
['WEWN', 'nacz']
same id at index: 3873
with id at: 3866
same id at index: 4185
with id at: 4177
['REUM', 'nefr']
same id at index: 4916
with id at: 4907
['NEFR', 'nech']
6852


In [7]:
print("table lenght {}".format(len(patient_data)))
print("classes array table len " + str( patient_classes.__len__()) )
print("dwa powtórzenia i 7 złączeń")

table lenght 6861
classes array table len 6852
dwa powtórzenia i 7 złączeń


In [8]:
bacteria_classes = [classes.Bacteria(data["drobnoustroj id"],
                                           data["id badania"],
                                           data["kod drobnoustroju"])
                                                        for index, data in bacteria_data.iterrows()]
drug_classes = [classes.DrugSusceptibility(data["id drobnoustroju"],
                                           data["kod leku"],
                                           data["opornosc"])
                                                        for index, data in drag_sus_data.iterrows()]

In [9]:
table_of_test_id = [klass.testId for klass in test_classes]
table_of_patient_id = [klass.PatientId for klass in patient_classes]

In [10]:
to_remove = []
for i in range(len(test_classes)):
    if test_classes[i].PatientId not in table_of_patient_id:
        to_remove.append(test_classes[i])
        
lenght_before = len(test_classes)
for o in to_remove:
    test_classes.remove(o)
len_after = len(test_classes)
print("badania filtrowanie: długość przed {} długość po {} delta {} ".format(lenght_before,len_after,lenght_before-len_after))
    
to_remove = []
for i in range(len(bacteria_classes)):
    if bacteria_classes[i].testId not in table_of_test_id:
        to_remove.append(bacteria_classes[i])
lenght_before = len(bacteria_classes)

for o in to_remove:
    bacteria_classes.remove(o)
    
len_after = len(bacteria_classes)
print("bakteria filtrowanie: długość przed {} długość po {} delta {} ".format(lenght_before,len_after,lenght_before-len_after))

badania filtrowanie: długość przed 16995 długość po 16995 delta 0 
bakteria filtrowanie: długość przed 15553 długość po 15421 delta 132 


In [11]:
to_remove = []

table_of_bacteria_id = [klass.bacteriaId for klass in bacteria_classes]
for i in range(len(drug_classes)):
    if drug_classes[i].DrugId not in table_of_bacteria_id:
        to_remove.append(drug_classes[i])
        
lenght_before = len(drug_classes)        

for o in to_remove:
    try:
        drug_classes.remove(o)    
    except ValueError as e:
        print(str(e))
len_after = len(drug_classes)
print("drug filtrowanie: długość przed {} długość po {} delta {} ".format(lenght_before,len_after,lenght_before-len_after))

drug filtrowanie: długość przed 136286 długość po 135220 delta 1066 


In [12]:
#budowanie drzewa zależności od antybiotyku w górę
for drug in tqdm(drug_classes):
    for i in range(len(bacteria_classes)):
        if drug.DrugId == bacteria_classes[i].bacteriaId:
            bacteria_classes[i].add_susceptibility(drug)
            break

100%|██████████| 135220/135220 [04:15<00:00, 529.97it/s]


In [13]:
bez_badania = 0
z_badaniem = 0
for bacteria in bacteria_classes:
    if bacteria.DrugSusceptibility:
        z_badaniem+=1
    else:
        bez_badania+=1
print("z badaniem {} bez {}".format(z_badaniem,bez_badania))

z badaniem 13505 bez 1916


In [14]:
#bakterie do badania - help :(
for bakteria in tqdm(bacteria_classes):
    for i in range(len(test_classes)):
        if bakteria.testId == test_classes[i].testId:
            test_classes[i].add_bacteria(bakteria)
            break

100%|██████████| 15421/15421 [00:57<00:00, 266.22it/s]


In [15]:
bez_bakteri = 0
z_bakterio = 0
for test in test_classes:
    if test.Bacteria:
        z_bakterio+=1
    else:
        bez_bakteri+=1
print("z bakterią {} bez {}".format(z_badaniem,bez_badania))

z bakterią 13505 bez 1916


In [16]:
#badania do pacjętów - there is no god he left us all
for test in tqdm(test_classes):
    for i in range(len(patient_classes)):
        if test.PatientId == patient_classes[i].PatientId:
            patient_classes[i].add_test(test)
            break

100%|██████████| 16995/16995 [00:08<00:00, 1918.27it/s]


In [17]:
bez_badania= 0
z_badaniem = 0
for patient in patient_classes:
    if patient.test:
        z_badaniem+=1
    else:
        bez_badania+=1
print("z badaniem {} bez {}".format(z_badaniem,bez_badania))

z badaniem 6746 bez 106


In [18]:
temp = [klass for klass in  patient_classes if klass.test and klass.test[0].materialId == 120]
#120 pochwa i z tego wychodzą same 0 czli 0-kob 1-mężcz

In [19]:
#ile materiałów testowano
len(test_data["kod materialu"].drop_duplicates())

41

In [20]:
#ile drobnoustrojów wykryto
len(bacteria_data["kod drobnoustroju"].drop_duplicates() )

178

In [21]:
#ile leków testowano
len( drag_sus_data["kod leku"].drop_duplicates() )

75

In [29]:
import importlib
import helper


importlib.reload(helper)

<module 'helper' from '/home/piernik/programing/python/jupiter/helper.py'>

In [30]:
helper.drugi_akapit(patient_classes,time_max=datetime.date(100,1,1))
helper.trzeci_akapit(patient_classes,time_max=datetime.date(1993,1,1))

(5108, 2981, 2328)

In [31]:
kobiety = [klass for klass in patient_classes if klass.sex == 0]
chłopy = [klass for klass in patient_classes if klass.sex == 1]

#drugi akapitp
print("wszystkie lata")
print( helper.drugi_akapit(patient_classes) ) #wspólne
print( helper.drugi_akapit(kobiety) )
print( helper.drugi_akapit(chłopy) )

print("co rok")
for i in range(1992,1997):
    print("rok " + str(i))
    print( helper.drugi_akapit(patient_classes, time_min= datetime.date(i,1,1), time_max=datetime.date(i,12,31))) #wspólne
    print( helper.drugi_akapit(kobiety,time_min= datetime.date(i,1,1), time_max=datetime.date(i,12,31) ))
    print( helper.drugi_akapit(chłopy,time_min= datetime.date(i,1,1), time_max=datetime.date(i,12,31) ))

wszystkie lata
(6746, 6746, 5189)
(3165, 3165, 2428)
(3581, 3581, 2761)
co rok
rok 1992
(2150, 2150, 1599)
(958, 958, 698)
(1192, 1192, 901)
rok 1993
(1367, 1367, 1282)
(690, 690, 630)
(677, 677, 652)
rok 1994
(1350, 1350, 1173)
(646, 646, 563)
(704, 704, 610)
rok 1995
(1864, 1864, 1254)
(888, 888, 613)
(976, 976, 641)
rok 1996
(559, 559, 405)
(258, 258, 191)
(301, 301, 214)


In [32]:
import collections
collections.Counter(patient_data["depName"].values)

print("we have 8 fields without department names")

we have 8 fields without department names


In [33]:
oddzial_list = list( patient_data["depName"].drop_duplicates().values)

oddzial_list.remove(numpy.nan)

oddzial_list = pd.DataFrame ([a.upper() for a in oddzial_list]).drop_duplicates()

oddzial_list = list(oddzial_list[0])

oddzial_list



['OIOM',
 'NACZ',
 'URAZ',
 'WEWN',
 'CHIR',
 'REUM',
 'NEFR',
 'UROL',
 'NECH',
 'NEUR',
 'USPR',
 'PLAS',
 'OKUL',
 'LARY',
 'SZCZ']

In [34]:
#przenieś wszystkie departmenty na all caps
for i in range(len(patient_classes)):
    for j in range(len(patient_classes[i].department)):
        try:
            patient_classes[i].department[j]=patient_classes[i].department[j].upper()
        except AttributeError as e:
            pass

In [50]:
#trzeci akapit
print("wszystkie lata")
print( helper.trzeci_akapit(patient_classes) ) #wspólne

helpe = helper.trzeci_akapit(patient_classes, time_min= datetime.date(200,1,1), time_max=datetime.date(2100,12,31))
print("{} &{} &{}& {} \\\\".format("Wspólnie",helpe[0],helpe[1],helpe[2]))  
helpe = helper.trzeci_akapit(chłopy, time_min= datetime.date(200,1,1), time_max=datetime.date(2100,12,31))
print("{} &{} &{}& {} \\\\".format("Mężczyźni",helpe[0],helpe[1],helpe[2]))
helpe = helper.trzeci_akapit(kobiety, time_min= datetime.date(200,1,1), time_max=datetime.date(2100,12,31))
print("{} &{} &{}& {} \\\\".format("Kobiety",helpe[0],helpe[1],helpe[2]))
print("\n\n")

print(r"\multicolumn{4}{|c|}{Wspólnie} \\")
for i in range(1992,1997):
    helpe = helper.trzeci_akapit(patient_classes, time_min= datetime.date(i,1,1), time_max=datetime.date(i,12,31))
    print("{} &{} &{}& {} \\\\ \\hline".format(str(i),helpe[0],helpe[1],helpe[2]))  #wspólne
print( "\n") 

print(r"\multicolumn{4}{|c|}{Mężczyźni} \\")
for i in range(1992,1997):
    helpe = helper.trzeci_akapit(chłopy, time_min= datetime.date(i,1,1), time_max=datetime.date(i,12,31))
    print("{} &{} &{}& {} \\\\ \\hline".format(str(i),helpe[0],helpe[1],helpe[2]))  #wspólne
print( "\n")

print(r"\multicolumn{4}{|c|}{Kobiety} \\")
for i in range(1992,1997):
    helpe = helper.trzeci_akapit(kobiety, time_min= datetime.date(i,1,1), time_max=datetime.date(i,12,31))
    print("{} &{} &{}& {} \\\\ \\hline".format(str(i),helpe[0],helpe[1],helpe[2]))  #wspólne
print( "\n")

for oddzial in oddzial_list:
    
    wydzial_classes = [klass for klass in  patient_classes if oddzial in  klass.department ]
    print(r"\hline \multicolumn{3}{|c|}{Oddział " + str(oddzial) + r"} \\ \hline")
    helpe = helper.trzeci_akapit(wydzial_classes)
    print("{} &{} &{} \\\\".format("wspólnie",helpe[1],helpe[2]))
    for i in range(1992,1997):
        #print("rok " + str(i))
        helpe = helper.trzeci_akapit(wydzial_classes, time_min= datetime.date(i,1,1), time_max=datetime.date(i,12,31))
        print("{} &{} &{} \\\\".format(str(i),helpe[1],helpe[2]))#wspólne

wszystkie lata
(16995, 10675, 9269)
Wspólnie &16995 &10675& 9269 \\
Mężczyźni &10295 &6377& 5657 \\
Kobiety &6700 &4298& 3612 \\



\multicolumn{4}{|c|}{Wspólnie} \\
1992 &5106 &2980& 2327 \\ \hline
1993 &3097 &2312& 1992 \\ \hline
1994 &3186 &2250& 2120 \\ \hline
1995 &4346 &2386& 2126 \\ \hline
1996 &1260 &747& 704 \\ \hline


\multicolumn{4}{|c|}{Mężczyźni} \\
1992 &3243 &1904& 1531 \\ \hline
1993 &1885 &1387& 1261 \\ \hline
1994 &1932 &1328& 1257 \\ \hline
1995 &2534 &1362& 1237 \\ \hline
1996 &701 &396& 371 \\ \hline


\multicolumn{4}{|c|}{Kobiety} \\
1992 &1863 &1076& 796 \\ \hline
1993 &1212 &925& 731 \\ \hline
1994 &1254 &922& 863 \\ \hline
1995 &1812 &1024& 889 \\ \hline
1996 &559 &351& 333 \\ \hline


\hline \multicolumn{3}{|c|}{Oddział OIOM} \\ \hline
wspólnie &3720 &3546 \\
1992 &958 &893 \\
1993 &823 &785 \\
1994 &878 &843 \\
1995 &808 &778 \\
1996 &253 &247 \\
\hline \multicolumn{3}{|c|}{Oddział NACZ} \\ \hline
wspólnie &327 &304 \\
1992 &115 &99 \\
1993 &100 &97 \\
1994 

In [28]:
#pierwszy grupowego
importlib.reload(helper)

helpe = helper.grup1(patient_classes)
print("{} &{} &{} &{}\\\\ \\hline".format(str("Wspólnie"),helpe[0],helpe[1],helpe[2]))  #wspólne
helpe = helper.grup1(chłopy)
print("{} &{} &{} &{}\\\\ \\hline".format(str("Mężczyźni"),helpe[0],helpe[1],helpe[2]))  #wspólne
helpe = helper.grup1(kobiety)
print("{} &{} &{} &{}\\\\ \\hline".format(str("Kobiety"),helpe[0],helpe[1],helpe[2]))  #wspólne


print("\n\n")
print(r"\multicolumn{4}{|c|}{Wspólnie} \\ \hline")
for i in range(1992,1997):
    #print(r"\multicolumn{3}{|c|}{Rok " + str(rok) + "}")
    helpe = helper.grup1(patient_classes, time_min= datetime.date(i,1,1), time_max=datetime.date(i,12,31))
    print("{} &{} &{} &{}\\\\ \\hline".format(str(i),helpe[0],helpe[1],helpe[2]))  #wspólne
print( "\n\n")

print(r"\multicolumn{4}{|c|}{Mężczyźni} \\ \hline")
for i in range(1992,1997):
    #print(r"\multicolumn{3}{|c|}{Rok " + str(rok) + "}")
    helpe = helper.grup1(chłopy, time_min= datetime.date(i,1,1), time_max=datetime.date(i,12,31))
    print("{} &{} &{} &{}\\\\ \\hline".format(str(i),helpe[0],helpe[1],helpe[2]))  #wspólne
print( "\n\n")

print(r"\multicolumn{4}{|c|}{Kobiety} \\ \hline")
for i in range(1992,1997):
    #print(r"\multicolumn{3}{|c|}{Rok " + str(rok) + "}")
    helpe = helper.grup1(kobiety, time_min= datetime.date(i,1,1), time_max=datetime.date(i,12,31))
    print("{} &{} &{} &{}\\\\ \\hline".format(str(i),helpe[0],helpe[1],helpe[2]))  #wspólne
print( "\n\n")

Wspólnie &0 &0 &0\\ \hline
Mężczyźni &0 &0 &0\\ \hline
Kobiety &0 &0 &0\\ \hline



\multicolumn{4}{|c|}{Wspólnie} \\ \hline
1992 &0 &0 &0\\ \hline
1993 &0 &0 &0\\ \hline
1994 &0 &0 &0\\ \hline
1995 &0 &0 &0\\ \hline
1996 &0 &0 &0\\ \hline



\multicolumn{4}{|c|}{Mężczyźni} \\ \hline
1992 &0 &0 &0\\ \hline
1993 &0 &0 &0\\ \hline
1994 &0 &0 &0\\ \hline
1995 &0 &0 &0\\ \hline
1996 &0 &0 &0\\ \hline



\multicolumn{4}{|c|}{Kobiety} \\ \hline
1992 &0 &0 &0\\ \hline
1993 &0 &0 &0\\ \hline
1994 &0 &0 &0\\ \hline
1995 &0 &0 &0\\ \hline
1996 &0 &0 &0\\ \hline





In [29]:
# Drógi punkt
nefr_classes = [klass for klass in  patient_classes if "NEFR" in  klass.department ]
nefr_kob_classes = [klass for klass in nefr_classes if klass.sex == 0 ]
nefr_chł_classes = [klass for klass in nefr_classes if klass.sex == 1 ]


for oddzial in oddzial_list:
    wydzial_classes = [klass for klass in  patient_classes if oddzial in  klass.department ]
    temp = helper.grup1(wydzial_classes)
    print("\\item {}:{}".format(oddzial,temp[0]))
print( "\n\n")    

for i in range(1992,1997):
    #print(r"\multicolumn{3}{|c|}{Rok " + str(rok) + "}")
    tmp_wsz = helper.grup1(nefr_classes, time_min= datetime.date(i,1,1), time_max=datetime.date(i,12,31))
    tmp_kob = helper.grup1(nefr_kob_classes, time_min= datetime.date(i,1,1), time_max=datetime.date(i,12,31))
    tmp_chł = helper.grup1(nefr_chł_classes, time_min= datetime.date(i,1,1), time_max=datetime.date(i,12,31))
    
    print("{} &{} &{} &{}\\\\ \\hline".format(str(i),tmp_wsz[0],tmp_kob[0],tmp_chł[0]))  #wspólne
print( "\n\n")

\item OIOM:0
\item NACZ:0
\item URAZ:0
\item WEWN:0
\item CHIR:0
\item REUM:0
\item NEFR:0
\item UROL:0
\item NECH:0
\item NEUR:0
\item USPR:0
\item PLAS:0
\item OKUL:0
\item LARY:0
\item SZCZ:0



1992 &0 &0 &0\\ \hline
1993 &0 &0 &0\\ \hline
1994 &0 &0 &0\\ \hline
1995 &0 &0 &0\\ \hline
1996 &0 &0 &0\\ \hline





In [30]:
#trzecie grupowe
importlib.reload(helper)

#sorted( dicte )

dicte = helper.grup3(patient_classes)
tab = sorted( dicte,key=lambda a:dicte[a],reverse=True )

for i in tab:
    print("{}:{}".format(i,dicte[i]))

18:1115
1:432
327:332
100:294
202:285
500:268
35:187
65:165
394:106
33:86
660:83
40:76
141:71
140:64
48:60
521:50
200:49
62:46
203:33
55:32
41:29
38:28
5:25
4:24
3:23
541:22
34:21
301:19
145:17
309:15
66:15
72:15
6:13
507:9
102:8
518:8
118:7
399:6
396:6
317:5
50:5
2:5
105:5
207:5
51:4
599:4
328:4
520:3
300:3
659:3
398:3
36:2
142:2
129:2
64:2
330:2
99:2
121:2
505:2
39:1
124:1
222:1
156:1
655:1
218:1
699:1
666:1
63:1
213:1
69:1
395:1
17:1
527:1
670:1
112:1
219:1
210:1
154:1
215:1
21:1
426:1
427:1
37:1
135:1
503:1
310:1
120:1


In [31]:
#18 cola 
def bakt(nr):
    print("\\hline Rok &Wspólnie & Kobiety & Mężczyźni \\\\ \\hline")
    tmp_wsz = helper.grup3_2(patient_classes,nr)
    tmp_kob = helper.grup3_2(kobiety,nr)
    tmp_chł = helper.grup3_2(chłopy,nr)
    print("Każdy& {}& {}& {} \\\\ \\hline ".format(tmp_wsz,tmp_kob,tmp_chł))
    
    for i in range(1992,1997):
        tmp_wsz = helper.grup3_2(patient_classes,nr, time_min= datetime.date(i,1,1), time_max=datetime.date(i,12,31))
        tmp_kob = helper.grup3_2(kobiety,nr , time_min= datetime.date(i,1,1), time_max=datetime.date(i,12,31))
        tmp_chł = helper.grup3_2(chłopy,nr, time_min= datetime.date(i,1,1), time_max=datetime.date(i,12,31))
        print("{}& {}& {}& {} \\\\ \\hline".format(i , tmp_wsz,tmp_kob,tmp_chł))

In [32]:
bakt(18)

\hline Rok &Wspólnie & Kobiety & Mężczyźni \\ \hline
Każdy& 1110& 823& 287 \\ \hline 
1992& 247& 169& 78 \\ \hline
1993& 254& 197& 57 \\ \hline
1994& 254& 196& 58 \\ \hline
1995& 277& 199& 78 \\ \hline
1996& 78& 62& 16 \\ \hline


In [33]:
bakt(1)

\hline Rok &Wspólnie & Kobiety & Mężczyźni \\ \hline
Każdy& 432& 228& 204 \\ \hline 
1992& 103& 60& 43 \\ \hline
1993& 90& 50& 40 \\ \hline
1994& 109& 60& 49 \\ \hline
1995& 92& 34& 58 \\ \hline
1996& 37& 24& 13 \\ \hline


In [60]:
#4 zadanie 
importlib.reload(helper)
dicte = helper.grup4_1(patient_classes)
tab = sorted( dicte,key=lambda a:dicte[a],reverse=True )

for i in tab:
    print("\\item {} : {}".format(i,dicte[i]))


\item 12.0 : 1149
\item 27.0 : 1120
\item 59.0 : 837
\item 29.0 : 831
\item 56.0 : 771
\item 38.0 : 737
\item 18.0 : 704
\item 34.0 : 688
\item 42.0 : 675
\item 39.0 : 575
\item 9.0 : 572
\item 76.0 : 571
\item 74.0 : 513
\item 6.0 : 512
\item 32.0 : 488
\item 16.0 : 423
\item 78.0 : 419
\item 23.0 : 416
\item 40.0 : 390
\item 1.0 : 353
\item 8.0 : 342
\item 83.0 : 309
\item 70.0 : 303
\item 17.0 : 275
\item 122.0 : 222
\item 4.0 : 186
\item 71.0 : 181
\item 62.0 : 131
\item 5.0 : 84
\item 11.0 : 48
\item 37.0 : 44
\item 54.0 : 21
\item 102.0 : 19
\item 28.0 : 17
\item 19.0 : 12
\item 41.0 : 10
\item 20.0 : 8
\item 43.0 : 6
\item 33.0 : 5
\item 25.0 : 3
\item 45.0 : 3
\item 50.0 : 2
\item 60.0 : 2
\item 21.0 : 2
\item 44.0 : 2
\item 123.0 : 1
\item 107.0 : 1
\item 124.0 : 1
\item 14.0 : 1
\item 35.0 : 1
\item 46.0 : 1


In [61]:
#18 cola 
def drug(nr):
    print("\\hline Rok &Wspólnie & Kobiety & Mężczyźni \\\\ \\hline")
    tmp_wsz = helper.grup4_2(patient_classes,nr)
    tmp_kob = helper.grup4_2(kobiety,nr)
    tmp_chł = helper.grup4_2(chłopy,nr)
    print("Każdy& {}& {}& {} \\\\ \\hline ".format(tmp_wsz,tmp_kob,tmp_chł))
    
    for i in range(1992,1997):
        tmp_wsz = helper.grup4_2(patient_classes,nr, time_min= datetime.date(i,1,1), time_max=datetime.date(i,12,31))
        tmp_kob = helper.grup4_2(kobiety,nr , time_min= datetime.date(i,1,1), time_max=datetime.date(i,12,31))
        tmp_chł = helper.grup4_2(chłopy,nr, time_min= datetime.date(i,1,1), time_max=datetime.date(i,12,31))
        print("{}& {}& {}& {} \\\\ \\hline".format(i , tmp_wsz,tmp_kob,tmp_chł))

In [62]:
drug(12)

\hline Rok &Wspólnie & Kobiety & Mężczyźni \\ \hline
Każdy& 710& 511& 199 \\ \hline 
1992& 159& 106& 53 \\ \hline
1993& 154& 116& 38 \\ \hline
1994& 218& 166& 52 \\ \hline
1995& 106& 65& 41 \\ \hline
1996& 73& 58& 15 \\ \hline


In [63]:
drug(27)

\hline Rok &Wspólnie & Kobiety & Mężczyźni \\ \hline
Każdy& 744& 563& 181 \\ \hline 
1992& 138& 104& 34 \\ \hline
1993& 135& 101& 34 \\ \hline
1994& 192& 148& 44 \\ \hline
1995& 207& 153& 54 \\ \hline
1996& 72& 57& 15 \\ \hline


In [92]:
import pprint as pp
importlib.reload(helper)
dicte = helper.grup4_3(patient_classes)
maximux = []
for key in dicte:
    maks = max(zip(dicte[key].values(),dicte[key].keys()))
    print("{} {}".format(key,maks))
    maximux.append(max(zip(dicte[key].values(),dicte[key].keys())))

print("\n\n".format( ))
maximux.sort(key=lambda x:x[0])
maximux

29.0 (747, 12.0)
83.0 (297, 59.0)
12.0 (911, 27.0)
23.0 (385, 12.0)
38.0 (635, 12.0)
18.0 (602, 27.0)
16.0 (391, 12.0)
76.0 (493, 59.0)
70.0 (249, 38.0)
56.0 (624, 27.0)
27.0 (911, 12.0)
34.0 (633, 27.0)
59.0 (706, 12.0)
9.0 (552, 27.0)
78.0 (386, 59.0)
6.0 (444, 27.0)
4.0 (169, 32.0)
40.0 (351, 74.0)
17.0 (221, 74.0)
39.0 (519, 12.0)
32.0 (376, 12.0)
71.0 (175, 27.0)
74.0 (400, 12.0)
62.0 (125, 74.0)
1.0 (282, 12.0)
42.0 (558, 12.0)
122.0 (174, 12.0)
33.0 (5, 27.0)
37.0 (39, 27.0)
50.0 (2, 29.0)
5.0 (82, 38.0)
20.0 (8, 40.0)
8.0 (336, 27.0)
54.0 (18, 59.0)
28.0 (19, 27.0)
123.0 (1, 83.0)
60.0 (4, 1.0)
19.0 (9, 12.0)
102.0 (18, 27.0)
25.0 (3, 27.0)
45.0 (3, 12.0)
107.0 (1, 42.0)
11.0 (47, 38.0)
41.0 (10, 74.0)
43.0 (6, 74.0)
124.0 (1, 71.0)
21.0 (2, 71.0)
14.0 (2, 21.0)
35.0 (2, 21.0)
46.0 (2, 1.0)
44.0 (2, 122.0)





[(1, 83.0),
 (1, 42.0),
 (1, 71.0),
 (2, 29.0),
 (2, 71.0),
 (2, 21.0),
 (2, 21.0),
 (2, 1.0),
 (2, 122.0),
 (3, 27.0),
 (3, 12.0),
 (4, 1.0),
 (5, 27.0),
 (6, 74.0),
 (8, 40.0),
 (9, 12.0),
 (10, 74.0),
 (18, 59.0),
 (18, 27.0),
 (19, 27.0),
 (39, 27.0),
 (47, 38.0),
 (82, 38.0),
 (125, 74.0),
 (169, 32.0),
 (174, 12.0),
 (175, 27.0),
 (221, 74.0),
 (249, 38.0),
 (282, 12.0),
 (297, 59.0),
 (336, 27.0),
 (351, 74.0),
 (376, 12.0),
 (385, 12.0),
 (386, 59.0),
 (391, 12.0),
 (400, 12.0),
 (444, 27.0),
 (493, 59.0),
 (519, 12.0),
 (552, 27.0),
 (558, 12.0),
 (602, 27.0),
 (624, 27.0),
 (633, 27.0),
 (635, 12.0),
 (706, 12.0),
 (747, 12.0),
 (911, 27.0),
 (911, 12.0)]

In [93]:
print("\\hline Rok &Wspólnie & Kobiety & Mężczyźni \\\\ \\hline")
tmp_wsz = helper.grup4_4(patient_classes)
tmp_kob = helper.grup4_4(kobiety)
tmp_chł = helper.grup4_4(chłopy)
print("Każdy& {}& {}& {} \\\\ \\hline ".format(tmp_wsz,tmp_kob,tmp_chł))

for i in range(1992,1997):
    tmp_wsz = helper.grup4_4(patient_classes, time_min= datetime.date(i,1,1), time_max=datetime.date(i,12,31))
    tmp_kob = helper.grup4_4(kobiety, time_min= datetime.date(i,1,1), time_max=datetime.date(i,12,31))
    tmp_chł = helper.grup4_4(chłopy, time_min= datetime.date(i,1,1), time_max=datetime.date(i,12,31))
    print("{}& {}& {}& {} \\\\ \\hline".format(i , tmp_wsz,tmp_kob,tmp_chł))

\hline Rok &Wspólnie & Kobiety & Mężczyźni \\ \hline
Każdy& 581& 434& 147 \\ \hline 
1992& 133& 99& 34 \\ \hline
1993& 119& 89& 30 \\ \hline
1994& 177& 134& 43 \\ \hline
1995& 80& 55& 25 \\ \hline
1996& 72& 57& 15 \\ \hline


In [108]:
#piąte
importlib.reload(helper)
dicte = helper.grup5(patient_classes)

sor = sorted(dicte.values())
print(sor)
dicte

[1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 4, 4, 6, 6, 6, 8, 11, 17, 24, 27, 66, 1115]


{18: 1115,
 1: 66,
 100: 27,
 327: 24,
 35: 17,
 33: 6,
 301: 2,
 394: 8,
 5: 2,
 500: 6,
 659: 1,
 65: 11,
 38: 3,
 200: 1,
 48: 2,
 41: 2,
 102: 1,
 55: 1,
 62: 1,
 140: 4,
 660: 4,
 328: 1,
 40: 2,
 202: 6,
 34: 1,
 4: 1}

In [109]:
print("\\hline Rok &Wspólnie & Kobiety & Mężczyźni \\\\ \\hline")
tmp_wsz = helper.grup5_2(patient_classes)
tmp_kob = helper.grup5_2(kobiety)
tmp_chł = helper.grup5_2(chłopy)
print("Każdy& {}& {}& {} \\\\ \\hline ".format(tmp_wsz,tmp_kob,tmp_chł))

for i in range(1992,1997):
    tmp_wsz = helper.grup5_2(patient_classes, time_min= datetime.date(i,1,1), time_max=datetime.date(i,12,31))
    tmp_kob = helper.grup5_2(kobiety, time_min= datetime.date(i,1,1), time_max=datetime.date(i,12,31))
    tmp_chł = helper.grup5_2(chłopy, time_min= datetime.date(i,1,1), time_max=datetime.date(i,12,31))
    print("{}& {}& {}& {} \\\\ \\hline".format(i , tmp_wsz,tmp_kob,tmp_chł))

\hline Rok &Wspólnie & Kobiety & Mężczyźni \\ \hline
Każdy& 66& 43& 23 \\ \hline 
1992& 9& 5& 4 \\ \hline
1993& 12& 7& 5 \\ \hline
1994& 15& 10& 5 \\ \hline
1995& 17& 13& 4 \\ \hline
1996& 13& 8& 5 \\ \hline


In [110]:
print("\\hline Rok &Wspólnie & Kobiety & Mężczyźni \\\\ \\hline")
tmp_wsz = helper.grup5_2(nefr_classes)
tmp_kob = helper.grup5_2(nefr_kob_classes)
tmp_chł = helper.grup5_2(nefr_chł_classes)
print("Każdy& {}& {}& {} \\\\ \\hline ".format(tmp_wsz,tmp_kob,tmp_chł))

for i in range(1992,1997):
    tmp_wsz = helper.grup5_2(nefr_classes, time_min= datetime.date(i,1,1), time_max=datetime.date(i,12,31))
    tmp_kob = helper.grup5_2(nefr_kob_classes, time_min= datetime.date(i,1,1), time_max=datetime.date(i,12,31))
    tmp_chł = helper.grup5_2(nefr_chł_classes, time_min= datetime.date(i,1,1), time_max=datetime.date(i,12,31))
    print("{}& {}& {}& {} \\\\ \\hline".format(i , tmp_wsz,tmp_kob,tmp_chł))

\hline Rok &Wspólnie & Kobiety & Mężczyźni \\ \hline
Każdy& 10& 7& 3 \\ \hline 
1992& 1& 0& 1 \\ \hline
1993& 1& 1& 0 \\ \hline
1994& 4& 2& 2 \\ \hline
1995& 2& 2& 0 \\ \hline
1996& 2& 2& 0 \\ \hline
